<h1 id="heading"><center><span style="background-color:#5642C5; color:white ; display:fill;border-radius:5px; font-family:cursive"> 1. Data Preprocessing 📊🔬</span></center><a class="anchor-link" href="">¶</a></h1>

In [ ]:
# Constante
DATA_PATH = '/home/saitama/Project/first_endtoend_project/data/raw/survey_results_public.csv' 
EXPORT_PATH = "../data/processed/1_preprocessed_df.pkl"
JOB_LIST_PATH ="../data/raw/job_list.csv" 

REPLACE_DICT = {
    'Age1stCode': {'Younger than 5 years': 4, 'Older than 85': 86},
    'YearsCodePro': {'Less than 1 year': 0, 'More than 50 years': 51}, 
    'YearsCode':    {'Less than 1 year': 0, 'More than 50 years': 51},
    }
# Columns has multiple answers on each value
COLUMNS_NEED_SPLIT = ["DatabaseDesireNextYear", "DatabaseWorkedWith",
                      "DevType",
                      "LanguageWorkedWith", "LanguageDesireNextYear",
                      "NEWCollabToolsDesireNextYear", "NEWCollabToolsWorkedWith",
                      "OpSys", "PlatformDesireNextYear", "PlatformWorkedWith",
                      "UndergradMajor",
                      "MiscTechDesireNextYear", "MiscTechWorkedWith",
                      "WebframeDesireNextYear", "WebframeWorkedWith"]

# Columns need to be integer 
COLUMNS_HAVE_PROB = ["Age1stCode", "YearsCodePro", "YearsCode"]
COLUMNS_TO_USE = COLUMNS_HAVE_PROB + COLUMNS_NEED_SPLIT

COLUMNS_FOR_JOB_HUNT = ["NEWJobHuntResearch", "NEWJobHunt"]

***
<h3 id="heading"><span style="background-color:#cefffb; color:#1b4946 ; display:fill;border-radius:5px; font-family:cursive"> 📥 Import packages & data </span></h3>

In [ ]:
# Load necessary package 
import pandas as pd
import numpy as np
import pickle
import logging
import math
import random
import os

pd.options.display.max_rows = 50000

raw_df = pd.read_csv(DATA_PATH)

***
<h2 id="heading"><span style="background-color:#cefffb; color:#1b4946; display:fill;border-radius:5px; font-family:cursive"> 📢 Helper Function  </span></h2>

In [ ]:
# Function 
def split_answers(column_serie, separator=';'):
    """
    Split multiple answers in a single string 
    to a list of single strings each represnting a single answers 
    
    Parametres:
    * column_Serie (pd.Series): string serie with multiple answers
    * separator (String): an other string that 
    
    return: (pd.Serie) contain serie of list 
    """
    
    # Sub-function checking if they are multiple answers
    def multi_value_checker(pd_series, separator):
        """ Check if the answer can be splitted by the separator - Returns boolean """
        return pd_series.str.contains(pat=separator)
    
    # Sub-function split each answer
    def split_answer(pd_series, separator):
        """ Split the string value by the separator - return list """
        return pd_series.str.split(separator) 

    # Check if the column_serie is splitable
    val_splitable = multi_value_checker(column_serie, separator)
    if not val_splitable.any():
        return column_serie
        
    # If val_splitable.any() is True then split the column
    column_splited = split_answer(column_serie, separator)
                                  
    # Replace the empty value by an empty list
    null_answers_bool = column_splited.isnull()
    column_splited.loc[null_answers_bool] = column_splited.loc[null_answers_bool].apply(lambda x: [])
    
    return column_splited

***
<h2 id="heading"><span style="background-color:#cefffb; color:#1b4946; display:fill;border-radius:5px; font-family:cursive"> 🧮 Preprocessing </span></h2>

<h4 id="heading"><span style="background-color:#fcf3b9; color:#3d3710 ; display:fill;border-radius:5px; font-family:Georgia"> 📋 Preprocessing Data </span></h4>

In [ ]:
df = raw_df.copy()

<h4 id="heading"><span style="background-color:#fcf3b9; color:#3d3710 ; display:fill;border-radius:5px; font-family:Georgia">1. 🔨🪛 Split Multiple Value
</span></h4>

In [ ]:
COLUMNS_SPLIT = df.select_dtypes('object').columns.to_list()
for column in COLUMNS_SPLIT:
    df[column] = split_answers(df[column],';')

<h4 id="heading"><span style="background-color:#fcf3b9; color:#3d3710 ; display:fill;border-radius:5px; font-family:Georgia">2. 🔨🪛 Replace Values </span></h4>

In [ ]:
# Change string value by the dictionary REPLACE_DICT (Pandas Remap Values)
df[COLUMNS_HAVE_PROB] = df[COLUMNS_HAVE_PROB].replace(REPLACE_DICT).astype(np.float32)

# Replace null value
for column in COLUMNS_HAVE_PROB:
    df[column].fillna(df[column].mean(), inplace = True)


***
<h2 id="heading"><span style="background-color:#cefffb; color:#1b4946; display:fill;border-radius:5px; font-family:cursive"> 💱 Check the change  </span></h2>

In [ ]:
test_col = random.choice(df.columns)
n_sample = random.randint(0, len(df))
print(test_col + '\n'+'-'*20)
print('In The Modified Data :', df[test_col].iloc[n_sample])
print('In The Actuel Data   :', raw_df[test_col].iloc[n_sample])

***
<h2 id="heading"><span style="background-color:#cefffb; color:#1b4946; display:fill;border-radius:5px; font-family:cursive"> 📤 Exporting Pickle </span></h2>

In [ ]:
EXPORT_PATH

In [ ]:
with open(EXPORT_PATH, 'wb') as f:
    pickle.dump(df, f)
# We can also use: $ df.to_pickle(EXPORT_PATH)

In [ ]:
# For import the pickle from the path
xr= pd.read_pickle(EXPORT_PATH)

In [ ]:
ROLE_COLS = []
for n in range(0,len(df)-1):
    ROLE_COLS += df['DevType'].iloc[n]

In [ ]:
JOB_LIST = np.unique(ROLE_COLS).tolist()

In [ ]:
jf = pd.DataFrame(JOB_LIST)
jf.to_csv(JOB_LIST_PATH)

In [ ]:
jf

### **🏅The index of person that  was has less than 5 years when they start coding**

In [ ]:
print("The index of person that  was has more than 85 years when they start coding:\n\n",
      raw_df[raw_df["Age1stCode"]=="Older than 85"].index.values)
print("--"*30)
print("The number of person that  was has more than 85 years when they start coding:-->",
      len(raw_df[raw_df["Age1stCode"]=="Older than 85"].index.values))

 1. **Display the row of Respondent who has more than 85 years when they start coding**

In [ ]:
 raw_df[raw_df["Age1stCode"]=="Older than 85"].sample(1).iloc[0]

### **🏅The index of person that  was has less than 5 years when they start coding**

In [ ]:
print("The index of person that  was has less than 5 years when they start coding:\n\n",
      raw_df[raw_df["Age1stCode"]=="Younger than 5 years"].index.values)
print("--"*30)
print("The number of person that  was has less than 5 years when they start coding:-->",
      len(raw_df[raw_df["Age1stCode"]=="Younger than 5 years"].index.values))

 1. **Display the row of Respondent who has more than 85 years when they start coding**

In [ ]:
 raw_df[raw_df["Age1stCode"]=="Younger than 5 years"].sample(1).iloc[0]

In [ ]:
print(raw_df.columns.tolist())

# 👨‍‍🔧 **Preprocessing**



## 🔬👓 Exploring Columns values

### 1️⃣ "MainBranch" 

In [ ]:
print(raw_df["MainBranch"].unique().tolist())

In [ ]:
# Change the value of the VALUE_TEST from any value of the  MainBranch column
# VALUE_TEST = "I am a developer by profession"
# VALUE_TEST = "I code primarily as a hobby"
VALUE_TEST = "I used to be a developer by profession, but no longer am"
# VALUE_TEST = "I am not primarily a developer, but I write code sometimes as part of my work"
# VALUE_TEST = "I am a student who is learning to code"
raw_df[raw_df["MainBranch"]==VALUE_TEST].sample(1).iloc[0]

In [ ]:
raw_df[raw_df["MainBranch"]==VALUE_TEST].shape
raw_df[raw_df["MainBranch"]==VALUE_TEST].describe()
raw_df[raw_df["MainBranch"]==VALUE_TEST].info()

In [ ]:
print(f"The mean age of coresponding that has '{VALUE_TEST}' is :",raw_df[raw_df["MainBranch"]==VALUE_TEST]["Age"].mean())

In [ ]:
raw_df